## Detokenize the tokens into coq proof

In [9]:
%load_ext autoreload
%autoreload 2

import pickle
tokenizer=pickle.load(open("Data/tokenizer.pkl", "rb"))
ruleDict=pickle.load(open("Data/ruleDict.pkl", "rb"))

encoding=pickle.load(open("mbjp/MBJP_826.pkl", "rb"))
print(len(encoding))
print(encoding)

tokens=tokenizer.convert_ids_to_tokens(encoding)
print(len(tokens))
print(tokens)

from program_model import detokenization
coq_proof=detokenization(tokens)
print(coq_proof.toString())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
186
[32217, 1564, 559, 951, 31506, 225, 32219, 482, 3845, 225, 32111, 1893, 559, 951, 31506, 225, 32220, 32221, 32100, 69, 225, 32220, 32221, 32100, 70, 225, 32220, 32221, 32100, 71, 225, 32222, 32220, 32220, 32223, 32109, 21253, 455, 225, 32224, 32109, 21253, 32225, 32222, 32220, 32252, 32228, 32229, 69, 225, 32229, 70, 225, 32220, 32235, 32230, 6923, 225, 32229, 455, 225, 32234, 32241, 10516, 88, 1202, 17, 539, 1259, 6251, 4341, 225, 32225, 32220, 32236, 32230, 10492, 225, 32229, 455, 225, 32225, 32222, 32252, 32237, 32229, 69, 225, 32229, 70, 225, 32220, 32235, 32230, 6923, 225, 32229, 455, 225, 32234, 32241, 9988, 624, 17, 539, 1259, 6251, 4341, 225, 32225, 32220, 32236, 32230, 10492, 225, 32229, 455, 225, 32225, 32222, 32232, 32237, 32229, 69, 225, 32229, 71, 225, 32220, 32235, 32230, 6923, 225, 32229, 455, 225, 32234, 32241, 4726, 17, 539, 1259, 6251, 4341, 225, 32225, 32220, 32236, 32230, 104

## Detokenizing Tokens into Proof

In [3]:
import subprocess, os, traceback
def validate_token2proof(id):
    if not os.path.exists("mbjp/MBJP_"+str(id)+".v"):
      return True
    try:
      encoding=pickle.load(open("mbjp/MBJP_"+str(id)+".pkl", "rb"))
      tokens=tokenizer.convert_ids_to_tokens(encoding)
      coq_proof=detokenization(tokens).toString()
    except Exception as e:
      print(f"Error in detokenizing proof {id}")
      return False
    prefix="""From PLF Require Import Syntax.
Open Scope string_scope.

Example prog_well_typed : exists p, program_well_typed p.
Proof.
unfold program_well_typed.
eexists.
eexists.
eapply T_ProgramConcat.
{ eapply T_ImportDecl with(n1:="java.io")(n2:="*"). }
{ eapply T_ProgramConcat.
  { eapply T_ImportDecl with(n1:="java.lang")(n2:="*"). }
  { eapply T_ProgramConcat.
    { eapply T_ImportDecl with(n1:="java.util")(n2:="*"). }
    { eapply T_ProgramConcat.
      { eapply T_ImportDecl with(n1:="java.math")(n2:="*"). }"""
    suffix=""" } } }
  Unshelve.
  all: apply STyVoid.
Defined.
Definition prog := the_exists_term (prog_well_typed).
Print prog."""
    coqProof=f"{prefix}\n{coq_proof}{suffix}"
    with open("../coq_code/test.v", "w") as file:
        file.write(coqProof)
    res = subprocess.run(
                # coqc -Q ../coq_code PLF ../coq_code/test.v
                ["coqc", "-Q","../coq_code", "PLF", "../coq_code/test.v"],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=60,
            )
    if res.returncode!=0:
        print(f"Error after detokenizing proof {id}")
        print(res.stderr.decode())
        return False
    else:
        return True

validate_token2proof(27)

True

In [10]:
from tqdm import trange
error_cnt=0
for i in trange(1, 1000):
    if not validate_token2proof(i):
        error_cnt+=1
print(f"Error count: {error_cnt}")

  0%|          | 0/999 [00:00<?, ?it/s]

100%|██████████| 999/999 [13:21<00:00,  1.25it/s]

Error count: 0


## Translating Proofs into Java

In [ ]:
def validateProof2Java(id):
    if not os.path.exists("mbjp/MBJP_"+str(id)+".v"):
      return True
    try:
      encoding=pickle.load(open("mbjp/MBJP_"+str(id)+".pkl", "rb"))
      tokens=tokenizer.convert_ids_to_tokens(encoding)
      javaCode=detokenization(tokens).toJava().toString()
    except Exception as e:
      print(f"Error in detokenizing proof {id}")
      return False
    try:
        from mxeval.execution import check_correctness_java as check
        import json
        with open("mbjp.json", "r") as file:
            mbjp = json.load(file)
        mbjp_name=f"MBJP/{id}" #MBJP/1
        for data in mbjp:
            if data['task_id']==mbjp_name:
                result=check(data,javaCode,solution_complete=True)
                if result['passed']:
                    return True
                else:
                    print(f"\nError in parsing java file: {mbjp_name}")
                    print(javaCode)
                    print(result['result'])
                    return False
        return False
    except Exception as e:
        print(f"\nError in parsing java file: {mbjp_name}")
        print(e)
        return False